# Given any MILP make Benders' decomposition using provided hints

In [1]:
from gurobipy import *
import pandas as pd
import numpy as np

UB = 1e3

supply = [10.,30.,40.,20]
demand = [20.,50.,30.]
varcost = [[2.,3.,4.], [3.,2.,1.], [1.,4.,3.], [4.,5.,2.]]
fixedcost = [[10.,30.,20.], [10.,30.,20.], [10.,30.,20.], [10.,30.,20.]]
ns = len(supply)
nd = len(demand)

xup = {(i,j):min(supply[i], demand[j]) for i in range(ns) for j in range(nd)}

In [2]:
milp = Model('milp')
x0 = {(i,j):milp.addVar(0, UB, 0., GRB.CONTINUOUS, 'x[%d,%d]' % (i,j)) for i in range(ns) for j in range(nd)}
y0 = {(i,j):milp.addVar(0, 1, 0., GRB.BINARY, 'y[%d,%d]'%(i,j)) for i in range(ns) for j in range(nd)}
milp.update()
cons_supply = [milp.addConstr( sum([x0[(i,j)] for j in range(nd)]) <= supply[i])  for i in range(ns)]
cons_demand = [milp.addConstr( sum([x0[(i,j)] for i in range(ns)]) >= demand[j]) for j in range(nd)]
cons_bigM   = [milp.addConstr(  x0[(i,j)] <= xup[(i,j)]*y0[(i,j)]) for i in range(ns) for j in range(nd)]
f = fixedcost
c = varcost
milp.setObjective(sum([f[i][j]*y0[(i,j)] + c[i][j]*x0[(i,j)] for i in range(ns) for j in range(nd)]),
                  GRB.MINIMIZE)

In [3]:
milp.Params.OutputFlag = 0
milp.optimize()

### Using Benders

In [5]:
from dynamicme.decomposition import Decomposer
from dynamicme.callback_gurobi import cb_benders

decomposer = Decomposer(milp)
master,sub = decomposer.benders_decomp()
#master.Params.Presolve = 0
#master.Params.LazyConstraints = 1
master.Params.OutputFlag = 0
sub.Params.OutputFlag = 0

master.optimize(cb_benders)

Changed value of parameter Presolve to 0
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter LazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0


%load_ext line_profiler

%lprun -f decomposer.benders_decomp decomposer.benders_decomp()

In [7]:
%load_ext line_profiler

In [8]:
from dynamicme.decomposition import split_constraints

%lprun -f split_constraints split_constraints(milp)

In [9]:
from scipy.sparse import csr_matrix, csc_matrix

csr_matrix()

### Compare

In [13]:
ys2 = [y for y in master.getVars() if y.VType != 'C']
yfin = [y.X for y in ys2]
decomposer.update_subobj(yfin)
sub.Params.OutputFlag=0
sub.optimize()

In [14]:
xs2 = {r.ConstrName:r.Pi for r in sub.getConstrs()}

print('%15.10s%15.10s%15.10s%15.10s%15.10s' % ('x0','x_sub','Diff', 'c0', 'c_sub'))
for xj1 in x0.values():    
    xj2 = xs2[xj1.VarName]
    print('%15.3g%15.3g%15.3g' % (xj1.X, xj2, xj2-xj1.X))

             x0          x_sub           Diff             c0          c_sub
              0              0              0
              0              0              0
             20             20              0
              0              0              0
              0              0              0
              0              0              0
             20             20              0
             30             30              0
             20             20         -2e-05
              0              0              0
              0              0              0
             10             10              0
